# Addressing Demo
In this notebook the geocoder will be configured, input dataframe is created from the data at Snowflake using Snowflake Connector for Spark in Databricks, and geocodes will be executed using the data.
Before executing this notebook make sure to have completed executing all cells in Addressing Installation notebook.

In [ ]:
# Import the required PySpark Libraries
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder \
  .config("spark.sql.legacy.allowUntypedScalaUDF", True) \
  .getOrCreate()

In [ ]:
# addressingRootDBFS will be the same path specified in the installation script
addressingRootDBFS = "/addressing"

# where to write the generated preferences file
preferencesFileDBFS = f"{addressingRootDBFS}/addressing.yaml"

# These should not need to be modified
sdkLocationLocal = f"/dbfs{addressingRootDBFS}/sdk"
preferencesFileLocal = f"/dbfs{preferencesFileDBFS}"
dataLocationLocal = f"/dbfs{addressingRootDBFS}/data"
extractLocationLocal = "/precisely/data"

## Geocode Preferences

In [ ]:
dbutils.fs.put(preferencesFileDBFS,"""
---
config:
  default:
    preferences:
      returnAllInfo: true
      clientCoordSysName: "epsg:4326"
""", True)
dbutils.fs.head(preferencesFileDBFS, 1024*1024*10)

Wrote 105 bytes.
Out[3]: '\n---\nconfig:\n default:\n preferences:\n returnAllInfo: true\n clientCoordSysName: "epsg:4326"\n'

In [ ]:
# import OS module
import os
addressing_dir = [path for path in os.listdir(sdkLocationLocal) if path.startswith("spectrum-bigdata-addressing")][0]
python_lib_dir = f"{sdkLocationLocal}/{addressing_dir}/pyspark/sdk/lib"
python_zip_location = python_lib_dir + "/" + [path for path in os.listdir(python_lib_dir) if path.startswith("spectrum-bigdata-addressing-sdk-pyspark")][0]
print("SDK Location Local:", python_zip_location)
# Adding the Python SDK files to the SparkContext
sc.addPyFile(python_zip_location)

# Resources Location
resourcesLocationLocal = f"{sdkLocationLocal}/{addressing_dir}/resources/"
print("Resource Location Local:", resourcesLocationLocal)

# Import all the required classes
from addressing.DownloadManagerBuilder import DownloadManagerBuilder
from addressing.AddressingBuilder import AddressingBuilder
from addressing.HDFSDownloader import HDFSDownloader
from addressing.S3Downloader import S3Downloader
from addressing.LocalFilePassthroughDownloader import LocalFilePassthroughDownloader
from addressing.PreferencesBuilder import PreferencesBuilder
from addressing.HadoopConfiguration import HadoopConfiguration
from addressing.UDFExecutor import UDFExecutor

# Create the Geocode UDF
geocodeUdf = AddressingBuilder() \
  .withResourcesLocation(resourcesLocationLocal) \
  .withDataLocations(dataLocationLocal) \
  .withExtractionLocation(extractLocationLocal) \
  .udfBuilder() \
  .withPreferencesFile(preferencesFileLocal) \
  .withOutputFields("customFields['PB_KEY']", "address.formattedStreetAddress", "address.formattedLocationAddress", "location.feature.geometry.coordinates.x", "location.feature.geometry.coordinates.y") \
  .withErrorField("error") \
  .forGeocode()

SDK Location Local: /dbfs/addressing/sdk/spectrum-bigdata-addressing-5.1.1.0/pyspark/sdk/lib/spectrum-bigdata-addressing-sdk-pyspark-5.1.1.0.zip
Resource Location Local: /dbfs/addressing/sdk/spectrum-bigdata-addressing-5.1.1.0/resources/

## Create the DataFrame by using Snowflake Connector for Spark in Databricks

In [ ]:
# Use secrets DBUtil to get Snowflake credentials.
user = dbutils.secrets.get("data-warehouse", "USERNAME")
password = dbutils.secrets.get("data-warehouse", "PASSWORD")
 
# snowflake connection options
options = {
  "sfUrl": "YOUR_SNOWFLAKE_URL",
  "sfUser": user,
  "sfPassword": password,
  "sfDatabase": "YOUR_DATABASE",
  "sfSchema": "YOUR_SCHEMA",
  "sfWarehouse": "YOUR_WAREHOUSE"
}

# Read the data written by the previous cell back.
df = spark.read \
  .format("snowflake") \
  .options(**options) \
  .option("dbtable", "YOUR_SNOWFLAKE_TABLE") \
  .load()
df.show(5, truncate=False)

+-------+----------------------------+-----------------+-----+-----+-------------------------------------------------+------------------------+
ID |ADDRESSLINE1 |CITY |STATE|ZIP |ADDRESS |COUNTRY |
+-------+----------------------------+-----------------+-----+-----+-------------------------------------------------+------------------------+
6128640|1504 78TH STREET ST LOUIS MO|SAINT LOUIS |MO |63130|1504 78TH STREET ST LOUIS MO,SAINT LOUIS,MO,63130|UNITED STATES OF AMERICA|
6135491|SOUTH MAIN ST |LEXINGTON |NC |27292|SOUTH MAIN ST,LEXINGTON,NC,27292 |UNITED STATES OF AMERICA|
6137561|HUMPHREY COLLISEUM |MISSISSIPPI STATE|MS |39762|HUMPHREY COLLISEUM,MISSISSIPPI STATE,MS,39762 |UNITED STATES OF AMERICA|
6088100|4755 VOLLMER/CICERO |MATTESON |IL |60443|4755 VOLLMER/CICERO,MATTESON,IL,60443 |UNITED STATES OF AMERICA|
6092449|6262 ROUTE 83 STE 301 |WILLOWBROOK |IL |60527|6262 ROUTE 83 STE 301,WILLOWBROOK,IL,60527 |UNITED STATES OF AMERICA|
+-------+----------------------------+-----------------+-----+-----+-------------------------------------------------+------------------------+
only showing top 5 rows

## Execute the Geocode
Note: Due to the behavior of the spark query execution planner, the geocode function could be executed multiple times for every record.  Because the geocode operation can be computationally expensive, this should be avoided.  Adding a call to "df.persist()" directly after the geocode function should ensure that each record calls the geocode function only once.

In [ ]:
df = df.withColumn("result", 
                   UDFExecutor() \
                   .apply(geocodeUdf,
                          create_map(lit("addressLines[0]"), col("ADDRESS"),
                                     lit("country"), col("COUNTRY")))) \
        .persist() \
        .select("*", "result.*").drop("result")

df.show(5, truncate=False)

+-------+----------------------------+-----------------+-----+-----+-------------------------------------------------+------------------------+----------------------+------------------------------+--------------------------------+---------------------------------------+---------------------------------------+-----+
ID |ADDRESSLINE1 |CITY |STATE|ZIP |ADDRESS |COUNTRY |customFields['PB_KEY']|address.formattedStreetAddress|address.formattedLocationAddress|location.feature.geometry.coordinates.x|location.feature.geometry.coordinates.y|error|
+-------+----------------------------+-----------------+-----+-----+-------------------------------------------------+------------------------+----------------------+------------------------------+--------------------------------+---------------------------------------+---------------------------------------+-----+
6128640|1504 78TH STREET ST LOUIS MO|SAINT LOUIS |MO |63130|1504 78TH STREET ST LOUIS MO,SAINT LOUIS,MO,63130|UNITED STATES OF AMERICA|P0000DD9S6NB |1504 78TH ST |SAINT LOUIS, MO 63130-1220 |-90.337416 |38.679108 |null |
6135491|SOUTH MAIN ST |LEXINGTON |NC |27292|SOUTH MAIN ST,LEXINGTON,NC,27292 |UNITED STATES OF AMERICA|null |S MAIN ST |LEXINGTON, NC 27292 |-80.253428 |35.823905 |null |
6137561|HUMPHREY COLLISEUM |MISSISSIPPI STATE|MS |39762|HUMPHREY COLLISEUM,MISSISSIPPI STATE,MS,39762 |UNITED STATES OF AMERICA|null |STATE ST |STARKVILLE, MS 39759 |-88.773149 |33.447554 |null |
6088100|4755 VOLLMER/CICERO |MATTESON |IL |60443|4755 VOLLMER/CICERO,MATTESON,IL,60443 |UNITED STATES OF AMERICA|P00007UTVBNV |4755 VOLLMER RD |MATTESON, IL 60443-1272 |-87.731941 |41.527394 |null |
6092449|6262 ROUTE 83 STE 301 |WILLOWBROOK |IL |60527|6262 ROUTE 83 STE 301,WILLOWBROOK,IL,60527 |UNITED STATES OF AMERICA|P00007UU57VN |6262 KINGERY HWY STE 301 |WILLOWBROOK, IL 60527-2987 |-87.949284 |41.775229 |null |
+-------+----------------------------+-----------------+-----+-----+-------------------------------------------------+------------------------+----------------------+------------------------------+--------------------------------+---------------------------------------+---------------------------------------+-----+
only showing top 5 rows